In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('horse_data.csv')
df.columns = ['surgery', 'age', 'hospital_number', 'rectal_temperature', 'pulse', 'respiratory_rate', 'temperature_of_extremities', 'peripheral_pulse', 'mucous_membranes', 'capillary_refill_time', 'pain', 'peristalsis', 'abdominal_distension', 'nasogastric_tube', 'nasogastric_reflux', 'nasogastric_reflux_PH', 'rectal_examination', 'abdomen', 'packed_cell_volume', 'total_protein', 'abdominocentesis_appearance', 'abdomcentesis_total_protein', 'outcome', 'surgical_lesion', 'lesion_1', 'lesion_2', 'lesion_3', 'cp_data']

df = df.replace('?', np.nan)
df.head()

,surgery,age,hospital_number,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,peripheral_pulse,mucous_membranes,capillary_refill_time,...,packed_cell_volume,total_protein,abdominocentesis_appearance,abdomcentesis_total_protein,outcome,surgical_lesion,lesion_1,lesion_2,lesion_3,cp_data
0,1,1,534817,39.2,88,20,NaN,NaN,4,1,...,50,85,2,2,3,2,2208,0,0,2
1,2,1,530334,38.30,40,24,1,1,3,1,...,33.00,6.70,NaN,NaN,1,2,0,0,0,1
2,1,9,5290409,39.10,164,84,4,1,6,2,...,48.00,7.20,3,5.30,2,1,2208,0,0,1
3,2,1,530255,37.30,104,35,NaN,NaN,6,2,...,74.00,7.40,NaN,NaN,2,2,4300,0,0,2
4,2,1,528355,NaN,NaN,NaN,2,1,3,1,...,NaN,NaN,NaN,NaN,1,2,0,0,0,2


# 1. Базовые метрики

## Для количественных данных

In [2]:
# соберем все числовые столбцы в виде словаря:
qual_values = {'rectal_temperature': df.rectal_temperature, 'pulse': df.pulse, 'respiratory_rate': df.respiratory_rate, 'nasogastric_reflux_PH': df.nasogastric_reflux_PH, 'packed_cell_volume': df.packed_cell_volume, 'total_protein': df.total_protein, 'abdomcentesis_total_protein': df.abdomcentesis_total_protein}

df_metrics = pd.DataFrame()

for key, value in qual_values.items():
    value = pd.to_numeric(value, errors='coerce')
    min_ = value.min()
    max_ = value.max()
    range_ = round((max_ - min_), 2)
    mean_ = round(value.mean(), 2)
    median_ = value.median()
    mode_ = value.mode()[0]
    std_ = round(value.std(), 2)
    var_ = round(value.var(), 2)
    Q1 = value.quantile(0.25)
    Q3 = value.quantile(0.75)
    Q33 = round(value.quantile(0.33), 2)
    IQR = round((Q3 - Q1), 2)
    row = {'series': key, 'min': min_, 'max': max_, 'range': range_, 'mean': mean_, 'median': median_, 'mode': mode_, 'std': std_, 'var': var_, 'Q1': Q1, 'Q3': Q3, 'Q1/3': Q33, 'IQR': IQR}
    df_metrics = pd.concat([df_metrics, pd.DataFrame([row])])

# добавим столбец с нормальными значениями (из описания):
df_norm = pd.DataFrame({'norm': ['37.8', '20-40', '8-10', '3-4', '30-50', '6-7.5', '<2.5']}, index=[0,0,0,0,0,0,0])
df_metrics_ = pd.concat([df_metrics, df_norm], axis=1)

df_metrics_

,series,min,max,range,mean,median,mode,std,var,Q1,Q3,Q1/3,IQR,norm
0,rectal_temperature,35.4,40.8,5.4,38.17,38.20,38.0,0.73,0.54,37.8,38.5,37.90,0.7,37.8
0,pulse,30.0,184.0,154.0,71.93,64.00,48.0,28.68,822.57,48.0,88.0,52.00,40.0,20-40
0,respiratory_rate,8.0,96.0,88.0,30.43,24.00,20.0,17.68,312.52,18.0,36.0,20.00,18.0,8-10
0,nasogastric_reflux_PH,1.0,7.5,6.5,4.71,5.00,2.0,1.98,3.93,3.0,6.5,4.00,3.5,3-4
0,packed_cell_volume,23.0,75.0,52.0,46.30,45.00,37.0,10.44,108.96,38.0,52.0,40.00,14.0,30-50
0,total_protein,3.3,89.0,85.7,24.52,7.50,6.5,27.51,756.75,6.5,57.0,6.74,50.5,6-7.5
0,abdomcentesis_total_protein,0.1,10.1,10.0,3.02,2.25,2.0,1.97,3.88,2.0,3.9,2.00,1.9,<2.5


## Для качественных данных

In [3]:
# соберем столбцы с качественными данными (представлены не все):
quant_values = {'surgery': df.surgery, 'age': df.age, 'temperature_of_extremities': df.temperature_of_extremities, 'peripheral_pulse': df.peripheral_pulse, 'mucous_membranes': df.mucous_membranes, 'capillary_refill_time': df.capillary_refill_time, 'pain': df.pain, 'peristalsis': df.peristalsis, 'abdominal_distension': df.abdominal_distension, 'nasogastric_tube': df.nasogastric_tube, 'nasogastric_reflux': df.nasogastric_reflux, 'rectal_examination': df.rectal_examination, 'abdomen': df.abdomen, 'abdominocentesis_appearance': df.abdominocentesis_appearance,  'outcome': df.outcome}

df_cat = pd.DataFrame()
for key_cat, value_cat in quant_values.items():
    row = {'series': key_cat, 'count': int(value_cat.describe()[0]), 'unique': int(value_cat.describe()[1]), 'top': value_cat.describe()[2], 'freq':  int(value_cat.describe()[3])}
    df_cat = pd.concat([df_cat, pd.DataFrame([row])])
df_cat

,series,count,unique,top,freq
0,surgery,298,2,1,180
0,age,299,1,2.1773,1
0,temperature_of_extremities,243,4,3,108
0,peripheral_pulse,230,4,1,115
0,mucous_membranes,253,6,1,79
0,capillary_refill_time,267,3,1,188
0,pain,244,5,3,67
0,peristalsis,255,4,3,128
0,abdominal_distension,243,4,1,76
0,nasogastric_tube,196,3,2,102


## Краткое описание
Датафрейм характеризует огромный разброс значений. На это указывают высокие показатели range, std, var, IQR. Отклонение минимальных значений от нормы несущественно в отличие от максимальных значений, которые в некоторых случаях превышают нормальный показатель в 5-10 раз. Наличие экстремально высоких показателей сказывается на средних значениях, которые в большинстве случаев превышают норму. Однако расчет моды свидетельствует о том, что большая часть лошадей имела нормальные показания тела или близко к норме. Из 7 модальных показателей только 2 (pulse и respiratory_rate) превышают норму (48 против 40, 20 против 10), а остальные (respiratory_rate, nasogastric_reflux_PH, packed_cell_volume, total_protein, abdomcentesis_total_protein) в норме или почти в норме (rectal_temperature имеет отклонение от нормы 0.2). Вывод: в целом ядро данных представляют средние значения, близкие к норме здоровья лошадей, однако имеются экстремально высокие значения.

# 2. Выбросы

In [4]:
# посчитаем выбросы:
def count_outliers(qual_values):
    df_outliers = pd.DataFrame()
    for key, value in qual_values.items():
        value = pd.to_numeric(value, errors='coerce')
        Q1 = value.quantile(0.25)
        Q3 = value.quantile(0.75)
        IQR = round((Q3 - Q1), 2)  
        lower_bound = (Q1 - (1.5 * IQR))
        upper_bound = Q3 + (1.5 * IQR)
        remove_outliers = df[value.between(lower_bound, upper_bound, inclusive=True)].sort_values(key)
        df_drop = pd.concat([df, remove_outliers]).drop_duplicates(keep=False)
        drop_count = len(df_drop)
        row = {'series': key, 'число выбросов': drop_count}
        df_outliers = pd.concat([df_outliers, pd.DataFrame([row])])
    return df_outliers
count_outliers(qual_values)

,series,число выбросов
0,rectal_temperature,74
0,pulse,29
0,respiratory_rate,75
0,nasogastric_reflux_PH,246
0,packed_cell_volume,32
0,total_protein,33
0,abdomcentesis_total_protein,204


Скорее всего, выбросы имеют естественный характер. Обычно болезнь связывают с повышением показателей тела (пульса, температуры и пр.). В некоторых случаях на ухудщение здоровья могут указывать значения, более низкие по сравнению с нормой. В нашем наборе данных обнаружено большое число выбросов для nasogastric_reflux_PH и abdomcentesis_total_protein. Это связано со слабой заполненностью столбцов (82.3 % и 65.9 % соответственно) — (?почему-то?) в выбросы включаются и пустые значения. Как бы там ни было, поскольку выбросы имеют естественную природу, то их лучше оставить.  

# 3. Работа с пропусками и др.

## Повторы

In [5]:
# проверим уникальность лошадей: 
uni_horses = len(df.hospital_number.unique())
print('Количество неуникальных лошадей:', len(df) - uni_horses)

Количество неуникальных лошадей: 16


In [6]:
# найдем всех неуникальных лошадей:
df_1 = df.groupby('hospital_number').agg({'hospital_number': 'count', 'respiratory_rate': 'nunique', 'rectal_temperature': 'nunique', 'pulse': 'nunique', 'total_protein': 'nunique', 'outcome': 'nunique'})
df_1.rename(columns={'hospital_number': 'number_count'}, inplace=True)
df_1 = df_1.reset_index()
df_1 = df_1.loc[df_1.number_count == 2]

# будем считать лошадь уникальной, если у нее хотя бы два параметра уникальны:
def find_repeats(row):
    if row.respiratory_rate + row.rectal_temperature + row.pulse + row.total_protein + row.outcome <= 7: 
        return row.hospital_number
to_delete = df_1.apply(find_repeats, axis=1).to_list()

#  удалим повторы из датафрейма:
new_df = df
for n in to_delete:
    new_df = new_df.loc[df.hospital_number != n]
print('Размер датафрейма после удаления повторов:', len(new_df))

Размер датафрейма после удаления повторов: 269


## Странности

In [7]:
# у столбца age были замечены неадекватные значения — проверим:
new_df.age.value_counts()

1    245
9     24
Name: age, dtype: int64

In [8]:
# столбец может принимать значения 1 или 2 (да или нет), поэтому заменим 9 на 2:
new_df.age.replace(9, 2, inplace=True)

## Пропуски

In [9]:
# посмотрим на непустые значения:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 269 entries, 0 to 298
Data columns (total 28 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   surgery                      268 non-null    object
 1   age                          269 non-null    int64 
 2   hospital_number              269 non-null    int64 
 3   rectal_temperature           218 non-null    object
 4   pulse                        250 non-null    object
 5   respiratory_rate             219 non-null    object
 6   temperature_of_extremities   214 non-null    object
 7   peripheral_pulse             201 non-null    object
 8   mucous_membranes             224 non-null    object
 9   capillary_refill_time        238 non-null    object
 10  pain                         216 non-null    object
 11  peristalsis                  228 non-null    object
 12  abdominal_distension         214 non-null    object
 13  nasogastric_tube             168 no

In [10]:
# посчитаем процент пустых значений для всех столбцов: 
for col in new_df.columns:
    pct_missing = new_df[col].isnull().mean()
    print(f'{col} — {pct_missing :.1%}')

surgery — 0.4%
age — 0.0%
hospital_number — 0.0%
rectal_temperature — 19.0%
pulse — 7.1%
respiratory_rate — 18.6%
temperature_of_extremities — 20.4%
peripheral_pulse — 25.3%
mucous_membranes — 16.7%
capillary_refill_time — 11.5%
pain — 19.7%
peristalsis — 15.2%
abdominal_distension — 20.4%
nasogastric_tube — 37.5%
nasogastric_reflux — 37.9%
nasogastric_reflux_PH — 82.5%
rectal_examination — 36.8%
abdomen — 42.4%
packed_cell_volume — 10.0%
total_protein — 10.8%
abdominocentesis_appearance — 55.8%
abdomcentesis_total_protein — 66.5%
outcome — 0.4%
surgical_lesion — 0.0%
lesion_1 — 0.0%
lesion_2 — 0.0%
lesion_3 — 0.0%
cp_data — 0.0%


In [11]:
# удалим столбцы, в которых более 50 % пропусков:
new_df = new_df.drop(['abdomcentesis_total_protein'], axis=1)
new_df = new_df.drop(['nasogastric_reflux_PH'], axis=1)

In [12]:
# особый случай — abdominocentesis_appearance: в нем пропущено более половины значений, но в тексте он помечен как важный
# можно заполнить пропуски на основе косинусного сходства, представив каждую строку как вектор 
# но грубый перебор всех вариантов с вычислением произведения и длины векторов не очень эффективен (другой метод не знаю)
# удалим его
new_df = new_df.drop(['abdominocentesis_appearance'], axis=1)

In [13]:
# outcome заслуживает отдельного рассмотрения (выжила лошадь или нет)
# в столбце 1 пропуск, при этом > значений в норме или немного выше нормы; скорее всего, лошадь выжила (1). 
new_df.outcome.fillna(1, inplace=True)

In [14]:
# удалим строки, в которых пропущены более 70 % значений (т.е. не заполнены 20 ячеек из 28):
new_df.dropna(thresh=20, inplace=True)
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 213 entries, 0 to 297
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   surgery                     212 non-null    object
 1   age                         213 non-null    int64 
 2   hospital_number             213 non-null    int64 
 3   rectal_temperature          179 non-null    object
 4   pulse                       205 non-null    object
 5   respiratory_rate            179 non-null    object
 6   temperature_of_extremities  197 non-null    object
 7   peripheral_pulse            187 non-null    object
 8   mucous_membranes            205 non-null    object
 9   capillary_refill_time       208 non-null    object
 10  pain                        196 non-null    object
 11  peristalsis                 206 non-null    object
 12  abdominal_distension        201 non-null    object
 13  nasogastric_tube            161 non-null    object

In [15]:
# заменим пропуски в количественных столбцах медианой, а в качественных — модой:
qual_values = {'rectal_temperature': new_df.rectal_temperature, 'pulse': new_df.pulse, 'respiratory_rate': new_df.respiratory_rate, 'packed_cell_volume': new_df.packed_cell_volume, 'total_protein': new_df.total_protein}
for key, value in qual_values.items():
    value.fillna(value.median(), inplace=True)

quant_values = {'surgery': new_df.surgery, 'age': new_df.age, 'temperature_of_extremities': new_df.temperature_of_extremities, 'peripheral_pulse': new_df.peripheral_pulse, 'mucous_membranes': new_df.mucous_membranes, 'capillary_refill_time': new_df.capillary_refill_time, 'pain': new_df.pain, 'peristalsis': new_df.peristalsis, 'abdominal_distension': new_df.abdominal_distension, 'nasogastric_tube': new_df.nasogastric_tube, 'nasogastric_reflux': new_df.nasogastric_reflux, 'rectal_examination': new_df.rectal_examination, 'abdomen': new_df.abdomen}
for key, value in quant_values.items():
    value.fillna(value.mode()[0], inplace=True)

new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 213 entries, 0 to 297
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   surgery                     213 non-null    object
 1   age                         213 non-null    int64 
 2   hospital_number             213 non-null    int64 
 3   rectal_temperature          213 non-null    object
 4   pulse                       213 non-null    object
 5   respiratory_rate            213 non-null    object
 6   temperature_of_extremities  213 non-null    object
 7   peripheral_pulse            213 non-null    object
 8   mucous_membranes            213 non-null    object
 9   capillary_refill_time       213 non-null    object
 10  pain                        213 non-null    object
 11  peristalsis                 213 non-null    object
 12  abdominal_distension        213 non-null    object
 13  nasogastric_tube            213 non-null    object

In [ ]:
# посмотрим, как изменились количественные метрики:
df_metrics = pd.DataFrame()

for key, value in qual_values.items():
    value = pd.to_numeric(value, errors='coerce')
    min_ = value.min()
    max_ = value.max()
    range_ = round((max_ - min_), 2)
    mean_ = round(value.mean(), 2)
    median_ = value.median()
    mode_ = value.mode()[0]
    std_ = round(value.std(), 2)
    var_ = round(value.var(), 2)
    Q1 = value.quantile(0.25)
    Q3 = value.quantile(0.75)
    Q33 = round(value.quantile(0.33), 2)
    IQR = round((Q3 - Q1), 2)
    row = {'series': key, 'min': min_, 'max': max_, 'range': range_, 'mean': mean_, 'median': median_, 'mode': mode_, 'std': std_, 'var': var_, 'Q1': Q1, 'Q3': Q3, 'Q1/3': Q33, 'IQR': IQR}
    df_metrics = pd.concat([df_metrics, pd.DataFrame([row])])

# добавим столбец с нормальными значениями (из описания):
df_norm = pd.DataFrame({'norm': ['37.8', '20-40', '8-10', '30-50', '6-7.5']}, index=[0,0,0,0,0])
df_metrics_ = pd.concat([df_metrics, df_norm], axis=1)

df_metrics_

После очистки набора данных минимальные и максимальные значения для оставшихся столбцов почти не изменились. Средние значения отличаются от исходных в пределах трех десятых, за исключением respiratory_rate, который стал ниже на 1.14, что при размахе 88 несущественно. То же самое можно сказать и о медиане, которая для pulse, respiratory_rate и total_protein осталась прежней, а для rectal_temperature и packed_cell_volume отличается на 0.1 (при размахе 5.4) и 1.0 (при размахе 52) соответственно. Среднее квадратичное отклонение и дисперсия в основном слегка снизились, за исключением total_protein, для которого они немного повысились. Незначительными являются изменения и для квартилей. Самые большие изменения произошли в моде. Для всех параметров значения моды увеличились, однако относительно нормы распределение осталось тем же: за ее пределами лежат pulse и respiratory_rate, в норме — packed_cell_volume и total_protein, близко к норме — rectal_temperature. В целом радикальных изменений по сравнению с первоначальными расчетами в новом датафрейме нет.    

In [16]:
# ...и качественные:
quant_values = {'surgery': new_df.surgery, 'age': new_df.age, 'temperature_of_extremities': new_df.temperature_of_extremities, 'peripheral_pulse': new_df.peripheral_pulse, 'mucous_membranes': new_df.mucous_membranes, 'capillary_refill_time': new_df.capillary_refill_time, 'pain': new_df.pain, 'peristalsis': new_df.peristalsis, 'abdominal_distension': new_df.abdominal_distension, 'nasogastric_tube': new_df.nasogastric_tube, 'nasogastric_reflux': new_df.nasogastric_reflux, 'rectal_examination': new_df.rectal_examination, 'abdomen': new_df.abdomen, 'outcome': new_df.outcome}
df_cat = pd.DataFrame()
for key_cat, value_cat in quant_values.items():
    row = {'series': key_cat, 'count': int(value_cat.describe()[0]), 'unique': int(value_cat.describe()[1]), 'top': value_cat.describe()[2], 'freq':  int(value_cat.describe()[3])}
    df_cat = pd.concat([df_cat, pd.DataFrame([row])])
df_cat

,series,count,unique,top,freq
0,surgery,213,2,1,125
0,age,213,1,0.256461,1
0,temperature_of_extremities,213,4,3,99
0,peripheral_pulse,213,4,1,125
0,mucous_membranes,213,6,1,72
0,capillary_refill_time,213,3,1,161
0,pain,213,5,3,77
0,peristalsis,213,4,3,106
0,abdominal_distension,213,4,1,76
0,nasogastric_tube,213,3,2,134


Для качественных данных значение моды (top) изменилось количественно, но не качественно: набор топовых значений остался прежним. Из него видно, что примерно половина параметров описывают нормальное состояние здоровья лошади. И, к примеру, такой важный показатель, как abdominal_distension, чаще всего указывает на отсутствие патологии ('1'  — отсутствие абдоминального растяжения). Другая половина характеризует патологию средней степени. Правда, 132 выживших лошади из оставшихся 213 (62 %) не самый оптимистичный показатель. Интересно, что большинство лошадей (125, или 59 %) на момент исследования уже имели хирургическую операцию: значения top '5' для surgery и abdomen говорят именно об этом. Следует добавить, что 93 % лошадей были взрослыми. 

Таким образом, наш набор данных в основном представляют патологии, которые возникли у взрослых особей, уже подвергавшихся хирургическому вмешательству. Причиной колики у таких лошадей, скорее всего, явилась нерешенная ранее проблема (неудачная операция, например) или новая проблема, связанная со старой. Довольно высокий процент летальных исходов (38 %) может свидетельствовать и о тяжести болезни (например, если у лошади были экстремально высокие показатели), и о неоказанной своевременной или надлежащей помощи (в случаях легкой и средней патологии).        

In [22]:
# новый датафрейм целиком:
new_df.head(20)

,surgery,age,hospital_number,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,peripheral_pulse,mucous_membranes,capillary_refill_time,...,rectal_examination,abdomen,packed_cell_volume,total_protein,outcome,surgical_lesion,lesion_1,lesion_2,lesion_3,cp_data
0,1,1,534817,39.2,88,20,3,1,4,1,...,4,2,50,85,3,2,2208,0,0,2
1,2,1,530334,38.30,40,24,1,1,3,1,...,1,1,33.00,6.70,1,2,0,0,0,1
2,1,2,5290409,39.10,164,84,4,1,6,2,...,3,5,48.00,7.20,2,1,2208,0,0,1
4,2,1,528355,38.1,64,24,2,1,3,1,...,3,3,44,7.5,1,2,0,0,0,2
5,1,1,526802,37.90,48,16,1,1,1,1,...,3,5,37.00,7.00,1,1,3124,0,0,2
6,1,1,529607,38.1,60,24,3,1,1,1,...,3,4,44.00,8.30,2,1,2208,0,0,2
7,2,1,530051,38.1,80,36,3,4,3,1,...,3,5,38.00,6.20,3,1,3205,0,0,2
8,2,2,5299629,38.30,90,24,1,1,1,1,...,3,5,40.00,6.20,1,2,0,0,0,1
9,1,1,528548,38.10,66,12,3,3,5,1,...,2,5,44.00,6.00,1,1,2124,0,0,1
10,2,1,527927,39.10,72,52,2,1,2,1,...,4,4,50.00,7.80,1,1,2111,0,0,2
